In [2]:
#pip install gspread oauth2client

In [47]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_earned = client.open("frb-volley-game-stats").worksheet("player-earned")

# Fetch all records (rows) from the sheet
player_earned_data = player_earned.get_all_records()

# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_earned_data)

In [52]:
df

,date,match-id,match,set,player,ace,spike,tip,dump,block,kills,attack-earned,total-earned
0,08/12/2024,141108,frb-gentofte,1,Lasse,1,0,0,1,0,1,1,2
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0,0,0,0
2,08/12/2024,141108,frb-gentofte,1,Kristian,0,1,0,0,0,1,1,1
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0,0,0,0
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,29/09/2024,141063,gentofte-frb,5,Mads,0,0,0,0,0,0,0,0
328,29/09/2024,141063,gentofte-frb,5,Martin,0,5,0,0,0,5,5,5
329,29/09/2024,141063,gentofte-frb,5,Mikkel,0,0,0,0,0,0,0,0
330,29/09/2024,141063,gentofte-frb,5,Frederik,0,0,0,0,0,0,0,0


In [53]:
points_per_set = df.groupby(['player', 'match', 'set'])['total-earned'].sum().reset_index()
points_per_set

,player,match,set,total-earned
0,Alex,avk-frb,1,0
1,Alex,avk-frb,2,0
2,Alex,avk-frb,3,0
3,Alex,avk-frb,4,0
4,Alex,avk-frb,5,0
...,...,...,...,...
326,Vestbjerg,gentofte-frb,1,2
327,Vestbjerg,gentofte-frb,2,1
328,Vestbjerg,gentofte-frb,3,5
329,Vestbjerg,gentofte-frb,4,0


In [54]:
points_per_game = df.groupby(['player', 'match'])['total-earned'].sum().reset_index()
points_per_game

,player,match,total-earned
0,Alex,avk-frb,0
1,Alex,frb-dtu,0
2,Alex,frb-gentofte,0
3,Alex,frb-vli,0
4,Alex,gentofte-frb,0
...,...,...,...
67,Vestbjerg,avk-frb,1
68,Vestbjerg,frb-dtu,11
69,Vestbjerg,frb-gentofte,3
70,Vestbjerg,frb-vli,1


In [55]:
total_points = df.groupby('player')['total-earned'].sum().reset_index()
total_points

,player,total-earned
0,Alex,0
1,Ando,0
2,Bo,27
3,Boerme,43
4,Bosse,2
5,Frederik,0
6,Gustav,37
7,Kristian,21
8,Lasse,42
9,Mads,3


In [56]:
df_final = points_per_set.merge(points_per_game, on=['player', 'match'], suffixes=('_per_set', '_per_game'))
df_final = df_final.merge(total_points, on='player', suffixes=('', '_total'))
df_final

,player,match,set,total-earned_per_set,total-earned_per_game,total-earned
0,Alex,avk-frb,1,0,0,0
1,Alex,avk-frb,2,0,0,0
2,Alex,avk-frb,3,0,0,0
3,Alex,avk-frb,4,0,0,0
4,Alex,avk-frb,5,0,0,0
...,...,...,...,...,...,...
326,Vestbjerg,gentofte-frb,1,2,8,24
327,Vestbjerg,gentofte-frb,2,1,8,24
328,Vestbjerg,gentofte-frb,3,5,8,24
329,Vestbjerg,gentofte-frb,4,0,8,24


In [57]:
df_final.drop(columns='match', inplace=True)
df_final

,player,set,total-earned_per_set,total-earned_per_game,total-earned
0,Alex,1,0,0,0
1,Alex,2,0,0,0
2,Alex,3,0,0,0
3,Alex,4,0,0,0
4,Alex,5,0,0,0
...,...,...,...,...,...
326,Vestbjerg,1,2,8,24
327,Vestbjerg,2,1,8,24
328,Vestbjerg,3,5,8,24
329,Vestbjerg,4,0,8,24


In [58]:
total_points['points'] = total_points['total-earned']
total_points

,player,total-earned,points
0,Alex,0,0
1,Ando,0,0
2,Bo,27,27
3,Boerme,43,43
4,Bosse,2,2
5,Frederik,0,0
6,Gustav,37,37
7,Kristian,21,21
8,Lasse,42,42
9,Mads,3,3


In [59]:
points_per_game['points'] = points_per_game['total-earned']
points_per_game['max-earned'] = points_per_game.groupby('player')['points'].transform('sum')
points_per_game

,player,match,total-earned,points,max-earned
0,Alex,avk-frb,0,0,0
1,Alex,frb-dtu,0,0,0
2,Alex,frb-gentofte,0,0,0
3,Alex,frb-vli,0,0,0
4,Alex,gentofte-frb,0,0,0
...,...,...,...,...,...
67,Vestbjerg,avk-frb,1,1,24
68,Vestbjerg,frb-dtu,11,11,24
69,Vestbjerg,frb-gentofte,3,3,24
70,Vestbjerg,frb-vli,1,1,24


In [60]:
import json

In [61]:
# Convert DataFrame to JSON
data = total_points.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../top-scorer.json", "w") as f:
    json.dump(data, f, indent=4)

#total_points.to_csv("../all_seasons_frb.csv", index=False)

In [62]:
# Convert DataFrame to JSON
data = points_per_game.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../top-scorer-per-game.json", "w") as f:
    json.dump(data, f, indent=4)

#total_points.to_csv("../all_seasons_frb.csv", index=False)